In [1]:
# Notebook to slim the production files for the LPR
import os
import sys
import tables as tb
import numpy  as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import dask.dataframe as dd

import re

In [44]:
# input_file = "../data/nexus/LPR_Tl208_Ports/PORT_1a/nexus_100_208Tl.h5"
input_file = "/Users/mistryk2/Packages/nexus/custom_macros/NEXT100_eminus.h5"

h5 = tb.open_file(input_file, 'r')
## Configuration
config = pd.read_hdf(input_file, 'MC/configuration')

display(config)

pressure = config[ config["param_key"] == "/Geometry/Next100/pressure"].iloc[0]['param_value']
pressure = [float(num) if '.' in num else int(num) for num in pressure.split() if num.replace('.', '', 1).isdigit()]
pressure = f"{int(pressure[0])}_bar"

# port = config[ config["param_key"] == "/Generator/IonGenerator/region"].iloc[0]['param_value']
# port = port.replace(" ", "") # removes the white space in the string

step_size = config[ config["param_key"] == "/Geometry/Next100/max_step_size"].iloc[0]['param_value']
step_size = step_size.replace(" ", "")

print(pressure)
# print(port)
print(step_size)
del input_file, h5

,param_key,param_value
0,event_type,other
1,num_events,10000
2,saved_events,10000
3,interacting_events,10000
4,/PhysicsList/RegisterPhysics,G4EmStandardPhysics_option4
5,/PhysicsList/RegisterPhysics,G4DecayPhysics
6,/PhysicsList/RegisterPhysics,G4RadioactiveDecayPhysics
7,/PhysicsList/RegisterPhysics,G4StepLimiterPhysics
8,/nexus/RegisterGeometry,Next100
9,/nexus/RegisterGenerator,SingleParticleGenerator


5_bar
0.1mm


In [46]:
# Now load in the stack 
# directory_path = '../data/nexus/LPR_Tl208_Ports/PORT_1a/'
directory_path = '/Users/mistryk2/Packages/nexus/custom_macros/'

# ddf = dd.read_hdf(os.path.join(directory_path, '*.h5'), key = 'MC/particles')
ddf = dd.read_hdf(os.path.join(directory_path, 'NEXT100_eminus.h5'), key = 'MC/particles')
parts = ddf.compute()

parts = parts.drop(columns=['initial_volume',
                            'initial_momentum_x', 'initial_momentum_y', 'initial_momentum_z',
                            'final_momentum_x', 'final_momentum_y', 'final_momentum_z'])

display(parts)

# Same with the hits
# ddf = dd.read_hdf(os.path.join(directory_path, '*.h5'), key = 'MC/hits')
ddf = dd.read_hdf(os.path.join(directory_path, 'NEXT100_eminus.h5'), key = 'MC/hits')
hits = ddf.compute()
display(hits)


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,final_y,final_z,final_t,final_volume,kin_energy,length,creator_proc,final_proc
0,99,1,e-,1,0,0.000000,0.000000,593.700012,0.000000,-16.464205,-306.544220,863.235962,2.301758,ACTIVE,2.600000,637.729248,none,eIoni
1,99,15,e-,0,1,-15.092288,-305.717285,862.660278,2.136695,-15.091941,-305.252136,863.769653,2.155849,ACTIVE,0.046857,2.590357,eIoni,eIoni
2,99,14,gamma,0,1,-12.731240,-307.266357,862.686707,2.121908,-16.675760,-308.190308,863.445801,2.135657,ACTIVE,0.016992,4.121797,eBrem,phot
3,99,21,e-,0,14,-16.675760,-308.190308,863.445801,2.135657,-16.675682,-308.190338,863.445618,2.135734,ACTIVE,0.000021,0.000360,phot,eIoni
4,99,20,e-,0,14,-16.675760,-308.190308,863.445801,2.135657,-16.675716,-308.190094,863.445862,2.135714,ACTIVE,0.000044,0.000320,phot,msc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458862,10098,30,e-,0,16,76.217453,-111.234299,334.691528,1.131275,76.217552,-111.234123,334.691437,1.131292,ACTIVE,0.000551,0.000233,phot,msc
458863,10098,29,e-,0,16,76.217453,-111.234299,334.691528,1.131275,76.218483,-111.230255,334.699036,1.131578,ACTIVE,0.003735,0.032794,phot,eIoni
458864,10098,28,e-,0,16,76.217453,-111.234299,334.691528,1.131275,76.169777,-111.001877,334.695221,1.138128,ACTIVE,0.024674,0.917886,phot,eIoni
458865,10098,15,e-,0,3,85.421791,-94.458260,379.249237,0.969521,74.087357,-103.546783,364.301849,1.328054,ACTIVE,0.354875,74.698883,phot,eIoni


,event_id,x,y,z,time,energy,label,particle_id,hit_id
0,99,0.025732,-0.020798,593.760620,0.000234,0.000352,ACTIVE,1,0
1,99,0.041836,-0.033195,593.799622,0.000382,0.000025,ACTIVE,1,1
2,99,0.078633,-0.060953,593.888367,0.000720,0.000293,ACTIVE,1,2
3,99,0.112339,-0.085909,593.969238,0.001029,0.000308,ACTIVE,1,3
4,99,0.158563,-0.118668,594.077698,0.001442,0.000087,ACTIVE,1,4
...,...,...,...,...,...,...,...,...,...
876083,10098,7.540542,18.407202,577.763855,0.100000,0.000291,ACTIVE,2,18
876084,10098,7.553649,18.406250,577.755554,0.100308,0.002039,ACTIVE,2,19
876085,10098,7.545705,18.403696,577.750244,0.100537,0.003180,ACTIVE,2,20
876086,10098,7.544089,18.403791,577.747925,0.100640,0.001423,ACTIVE,2,21


In [47]:
# Open the HDF5 file in write mode
with pd.HDFStore(f"../data/nexus/LPR_Tl208_Ports//NEXT100_eminus_slim.h5", mode='w', complevel=5, complib='zlib') as store:
    # Write each DataFrame to the file with a unique key
    store.put('config', config, format='table')
    store.put('parts',parts, format='table')
    store.put('hits',hits, format='table')

In [11]:
# Now load in the stack 
directory_path = '../data/nexus/LPR_Tl208_Ports/PORT_1a/esmeralda/'

ddf = dd.read_hdf(os.path.join(directory_path, '*.h5'), key = 'MC/particles')
parts = ddf.compute()

display(parts)

# Same with the hits
ddf = dd.read_hdf(os.path.join(directory_path, '*.h5'), key = 'MC/hits')
hits = ddf.compute()
display(hits)

# Get the reconstructed information for low energy hits (lower threshold)
ddf = dd.read_hdf(os.path.join(directory_path, '*.h5'), key = '/CHITS/highTh')
ht_chits = ddf.compute()
display(ht_chits)



,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
0,990000,1,Tl208,True,0,398.875244,394.635712,532.951294,0.000000,398.875244,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,none,RadioactiveDecay
1,990000,4,e-,False,1,398.875244,394.635712,532.951294,0.000465,399.641754,...,0.496680,0.184480,-0.907411,0.000000,0.00000,0.000000,0.657435,1.859799,RadioactiveDecay,eIoni
2,990000,5,e-,False,4,399.025238,394.691437,532.677246,0.001642,399.041443,...,-0.020379,-0.023106,-0.019091,0.000000,0.00000,-0.000000,0.001284,0.103721,eIoni,eIoni
3,990000,3,anti_nu_e,False,1,398.875244,394.635712,532.951294,0.000465,-3993.610352,...,-0.275837,-0.809750,0.137137,-0.275837,-0.80975,0.137137,0.866364,13796.178711,RadioactiveDecay,Transportation
4,990000,2,Pb208[3475.078],False,1,398.875244,394.635712,532.951294,0.000465,398.875244,...,-0.220843,0.625269,0.770273,-0.000000,0.00000,0.000000,0.000003,0.000000,RadioactiveDecay,RadioactiveDecay
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22748,80338,109,gamma,False,104,203.234116,198.818390,691.732727,1.295985,203.255676,...,0.001539,-0.000522,0.002616,0.000000,-0.00000,0.000000,0.003080,0.043122,eBrem,phot
22749,80338,112,e-,False,109,203.255676,198.811081,691.769348,1.296129,203.255615,...,-0.003031,-0.005353,0.003005,-0.000000,-0.00000,0.000000,0.000046,0.000119,phot,msc
22750,80338,111,e-,False,109,203.255676,198.811081,691.769348,1.296129,203.255936,...,0.007892,-0.020303,-0.011175,0.000000,-0.00000,-0.000000,0.000586,0.003208,phot,eIoni
22751,80338,110,e-,False,109,203.255676,198.811081,691.769348,1.296129,203.259232,...,0.039186,0.029634,0.006858,-0.000000,0.00000,-0.000000,0.002402,0.019339,phot,eIoni


,event_id,particle_id,hit_id,x,y,z,time,energy,label
0,990000,7,0,354.250397,248.590759,314.866608,1.019044,0.000045,ACTIVE
1,990000,7,1,435.379852,214.032700,166.554489,1.594601,0.000174,ACTIVE
2,990000,23,0,435.379852,214.032700,166.554489,1.594602,0.000044,ACTIVE
3,990000,22,0,435.379883,214.032639,166.554642,1.594644,0.000047,ACTIVE
4,990000,21,0,435.379944,214.032715,166.554474,1.594632,0.000021,ACTIVE
...,...,...,...,...,...,...,...,...,...
98249,80338,111,0,203.255753,198.810883,691.769226,1.296146,0.000018,ACTIVE
98250,80338,111,1,203.255936,198.810394,691.768982,1.296188,0.000568,ACTIVE
98251,80338,110,0,203.257019,198.812103,691.769592,1.296188,0.000071,ACTIVE
98252,80338,110,1,203.259476,198.813965,691.770020,1.296297,0.002040,ACTIVE


,event,time,npeak,Xpeak,Ypeak,nsipm,X,Y,Xrms,Yrms,Z,Q,E,Qc,Ec,track_id,Ep
0,1980000,1.980001e+12,0,434.069955,206.608199,1,427.625,194.375,0.0,0.0,171.685226,45.391304,5500.944260,-1.0,0.021541,-1,0.021541
1,1980000,1.980001e+12,0,434.069955,206.608199,1,427.625,194.375,0.0,0.0,172.716980,56.869564,2937.036713,-1.0,0.011385,-1,0.011385
2,1980000,1.980001e+12,0,434.069955,206.608199,1,427.625,194.375,0.0,0.0,173.743103,53.739132,2993.497070,-1.0,0.011696,-1,0.011696
3,1980000,1.980001e+12,0,434.069955,206.608199,1,427.625,194.375,0.0,0.0,174.764053,35.768116,2935.002694,-1.0,0.011759,-1,0.011759
4,1980000,1.980001e+12,1,355.067975,244.194976,1,349.875,241.025,0.0,0.0,312.391235,32.000000,10500.999939,-1.0,0.036987,-1,0.036987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7614,160676,1.606765e+11,1,227.391575,239.144270,1,225.475,225.475,0.0,0.0,728.159058,30.086956,2900.331943,-1.0,0.009848,-1,0.009848
7615,160676,1.606765e+11,1,227.391575,239.144270,1,225.475,241.025,0.0,0.0,728.159058,32.405796,3123.864221,-1.0,0.010607,-1,0.010607
7616,160676,1.606765e+11,1,227.391575,239.144270,1,241.025,241.025,0.0,0.0,728.159058,31.710144,3056.804538,-1.0,0.010379,-1,0.010379
7617,160676,1.606765e+11,1,227.391575,239.144270,1,225.475,241.025,0.0,0.0,729.199219,30.260870,8307.999710,-1.0,0.028196,-1,0.028196
